In [1]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 使用demo

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的30字以内笑话")
model = ChatOpenAI()
output_parser = StrOutputParser()

# |表示前一个模块的输出作为后一个模块的输入
chain = prompt | model | output_parser

chain.invoke({"topic": "大头"})

'为什么大头的人总是戴着帽子？因为帽子是他们的头的加长款！哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈'

# prompt的各种数据形式

In [12]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于刺猬的30字以内笑话')])

In [14]:
prompt_value.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptValue'],
 'kwargs': {'messages': [HumanMessage(content='给我讲一个关于刺猬的30字以内笑话')]}}

In [15]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于刺猬的30字以内笑话')]

In [16]:
prompt_value.to_string()

'Human: 给我讲一个关于刺猬的30字以内笑话'

# model

In [18]:
message = model.invoke(prompt_value)
message

AIMessage(content='为什么刺猬不喜欢参加聚会？因为他总是被人围着问：“你的刺是怎么梳的？”', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 26, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d1275622-3112-4a2d-b9f1-1657ee8e6a83-0')

# 使用llm的区别

In [19]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\n刺猬问刺猬：“我们为什么有那么多刺？”另一个刺猬回答：“因为我们是刺猬啊！”'

# output_parser

In [20]:
output_parser.invoke(message)

'为什么刺猬不喜欢参加聚会？因为他总是被人围着问：“你的刺是怎么梳的？”'

# RAG Search Example
- 建立向量数据
- 使用RAG增强
<hr>

In [21]:
! pip install --upgrade --quiet  faiss-cpu tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
doctran-openai 0.0.21 requires tiktoken<0.6.0,>=0.5.2, but you have tiktoken 0.6.0 which is incompatible.


In [39]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.globals import set_debug

# set_verbose(True)
set_debug(False)

vectorstore = FAISS.from_texts(["bigHead worked at beijing"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
chain = (
    # 这里直接传递retriever不太对,prompt中放入的是Document对象,但是没有试出来正确的写法
    # RunnablePassthrough: 实现context的内容会添加到prompt中,不确定理解的是否正确
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)
chain.invoke("where did bigHead work?")

'bigHead worked at Beijing.'

In [40]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    # itemgetter : 从chain.invoke的参数中获取指定参数
        {"context": itemgetter("question") | retriever, "question": itemgetter("question"),
         "language": itemgetter("language")}
        | prompt
        | model
        | StrOutputParser()
)

In [46]:
chain.invoke({"question": "大头在哪里工作?", "language": "chinese"})

'大头在北京工作。'

# 输入输出格式

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")
chain = prompt | model

## input_schema

In [49]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [50]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [51]:
# 相对就比较复杂一些
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

## output_schema

In [52]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper